In [ ]:
# coding: utf-8
# 从压缩包中提取数据到csv文件
import os
import tarfile
import gzip
import zipfile
import csv
import re

tenhou_files_path = '../dataset/Tenhou'
files = os.listdir(tenhou_files_path)
pattern = re.compile(r'(\d{4})/scc')

for file in files:

    # 判断是否为压缩文件
    if not file.endswith('.zip'):
        continue

    with zipfile.ZipFile(os.path.join(tenhou_files_path, file), 'r') as zip_ref:

        namelist = zip_ref.namelist()
        extracted_data_filename = file[5:9] + ".csv"
        with open(os.path.join(tenhou_files_path, extracted_data_filename), 'w') as extracted_data_file:
            writer = csv.writer(extracted_data_file)
            writer.writerow(['date', 'time', 'game_rule', 'log_url'])

            for name in namelist:
                if name.endswith('.html.gz') and re.match(pattern, name):
                    year = name[-16:-12]
                    month = name[-12:-10]
                    day = name[-10:-8]
                    date = year + "/" + month + "/"+ day
                    print(date)

                    zip_ref.extract(name, os.path.join(tenhou_files_path, 'temp'))
                    with gzip.open(os.path.join(tenhou_files_path, 'temp', name), 'r') as html_file:
                        data_b = html_file.readlines()
                        data = [line.decode('utf-8').strip() for line in data_b]
                        # print(data[0])
                        # print(len(data))
                        # break
                        for i in range(len(data)):
                            split_data = data[i].split('|')
                            time = split_data[0].strip()
                            game_rule = split_data[2].strip()
                            raw_url = split_data[3].strip()
                            # print(split_data)
                            # break

                            try:
                                # 提取URL链接部分
                                start_index = raw_url.find('http')  # 找到URL的起始位置
                                end_index = raw_url.find('">')  # 找到URL的结束位置

                                if start_index != -1 and end_index != -1:
                                    url_part = raw_url[start_index:end_index]  # 提取URL部分

                                    # 替换"?log="为"log/?"
                                    modified_url = url_part.replace('?log=', 'log/?')

                                    # print(modified_url)  # 输出修改后的URL
                            
                            except:
                                modified_url = 'None'
                            
                            series = [date, time, game_rule, modified_url]
                            writer.writerow(series)


                if name.endswith('.html') and name.startswith(''):

                    year = name[-13:-9]
                    month = name[-9:-7]
                    day = name[-7:-5]
                    date = year + "/" + month + "/"+ day

                    with zip_ref.open(name, 'r') as html_file:
                        data_b = html_file.readlines()
                        data = [line.decode('utf-8').strip() for line in data_b]
                        # print(data[0])
                        # print(len(data))
                        # break
                        for i in range(len(data)):
                            split_data = data[i].split('|')
                            time = split_data[0].strip()
                            game_rule = split_data[2].strip()
                            raw_url = split_data[3].strip()
                            # print(split_data)
                            # break

                            try:
                                # 提取URL链接部分
                                start_index = raw_url.find('http')  # 找到URL的起始位置
                                end_index = raw_url.find('">')  # 找到URL的结束位置

                                if start_index != -1 and end_index != -1:
                                    url_part = raw_url[start_index:end_index]  # 提取URL部分

                                    # 替换"?log="为"log/?"
                                    modified_url = url_part.replace('?log=', 'log/?')

                                    # print(modified_url)  # 输出修改后的URL
                            
                            except:
                                modified_url = 'None'
                            
                            series = [date, time, game_rule, modified_url]
                            writer.writerow(series)

In [ ]:
# 筛选四鳳南喰赤的数据

import pandas as pd
import numpy as np
import os
import re



# 读取csv文件
tenhou_files_path = '../dataset/Tenhou'
files = os.listdir(tenhou_files_path)
pattern = re.compile(r'(\d{4})\.csv')

for file in files:
    if pattern.match(file):
        data = pd.read_csv(os.path.join(tenhou_files_path, file))
        nan_4_kuitan_aka = data[data['game_rule'].str.startswith('四鳳南喰赤')]
        print(file[:-4],"年总对局：")
        print(data.shape)
        print("四鳳南喰赤总对局：")
        print(nan_4_kuitan_aka.shape)
        nan_4_kuitan_aka.to_csv(os.path.join(tenhou_files_path, file[:-4] + '_nan_4_kuitan_aka.csv'), index=False)

In [ ]:
# 根据URL下载牌谱数据并保存

import urllib.request
import os
import pandas as pd
import gzip
import time

data = pd.read_csv('../dataset/Tenhou/2011_nan_4_kuitan_aka.csv')
HEADER = {
    'Host': 'e.mjv.jp',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive'
}

# 读取第一条数据
for index, url in data['log_url'].items():
    req = urllib.request.Request(url, headers=HEADER)
    opener = urllib.request.build_opener()
    response = opener.open(req)
    log = gzip.decompress(response.read()).decode('utf-8')
    time.sleep(0.1)

    # 保存到文件
    save_path = '../dataset/Tenhou/2011_nan_4_kuitan_aka'
    file_name = str(index) + '.log'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    with open(os.path.join(save_path, file_name), 'w') as f:
        f.write(log)

# print(log)

In [ ]:
# 将xml格式的牌谱数据转换为编码格式

import re
import os
from bs4 import BeautifulSoup

# 0~135表示136种牌
pai_dict = {
    # 0~35: 万
    0: '1m', 1: '1m', 2: '1m', 3: '1m', 4: '2m', 5: '2m', 6: '2m', 7: '2m', 8: '3m', 9: '3m', 10: '3m', 11: '3m',
    12: '4m', 13: '4m', 14: '4m', 15: '4m', 16: '0m', 17: '5m', 18: '5m', 19: '5m', 20: '6m', 21: '6m', 22: '6m',
    23: '6m', 24: '7m', 25: '7m', 26: '7m', 27: '7m', 28: '8m', 29: '8m', 30: '8m', 31: '8m', 32: '9m', 33: '9m',
    34: '9m', 35: '9m',
    # 36~71: 筒
    36: '1p', 37: '1p', 38: '1p', 39: '1p', 40: '2p', 41: '2p', 42: '2p', 43: '2p', 44: '3p', 45: '3p', 46: '3p',
    47: '3p', 48: '4p', 49: '4p', 50: '4p', 51: '4p', 52: '0p', 53: '5p', 54: '5p', 55: '5p', 56: '6p', 57: '6p',
    58: '6p', 59: '6p', 60: '7p', 61: '7p', 62: '7p', 63: '7p', 64: '8p', 65: '8p', 66: '8p', 67: '8p', 68: '9p',
    69: '9p', 70: '9p', 71: '9p',
    # 72~107: 条
    72: '1s', 73: '1s', 74: '1s', 75: '1s', 76: '2s', 77: '2s', 78: '2s', 79: '2s', 80: '3s', 81: '3s', 82: '3s',
    83: '3s', 84: '4s', 85: '4s', 86: '4s', 87: '4s', 88: '0s', 89: '5s', 90: '5s', 91: '5s', 92: '6s', 93: '6s',
    94: '6s', 95: '6s', 96: '7s', 97: '7s', 98: '7s', 99: '7s', 100: '8s', 101: '8s', 102: '8s', 103: '8s', 104: '9s',
    105: '9s', 106: '9s', 107: '9s',
    # 108~135: 字
    108: '1z', 109: '1z', 110: '1z', 111: '1z', 112: '2z', 113: '2z', 114: '2z', 115: '2z', 116: '3z', 117: '3z',
    118: '3z', 119: '3z', 120: '4z', 121: '4z', 122: '4z', 123: '4z', 124: '5z', 125: '5z', 126: '5z', 127: '5z',
    128: '6z', 129: '6z', 130: '6z', 131: '6z', 132: '7z', 133: '7z', 134: '7z', 135: '7z'
}
player_dict = {
    '0': '自家',
    '1': '下家',
    '2': '对家',
    '3': '上家'
}
kyoku_dict = {
    '0': '东一局',
    '1': '东二局',
    '2': '东三局',
    '3': '东四局',
    '4': '南一局',
    '5': '南二局',
    '6': '南三局',
    '7': '南四局'
}
draw_dict = {
    'T': '自家摸牌',
    'U': '下家摸牌',
    'V': '对家摸牌',
    'W': '上家摸牌'
}
discard_dict = {
    'D': '自家弃牌',
    'E': '下家弃牌',
    'F': '对家弃牌',
    'G': '上家弃牌'
}

class taikyoku(object):

    def __init__(self, file) -> None:
        self.file = file
        self.data = self._load_file(file) # 存放牌谱数据

        # 存放当前对局的信息
        self.info = {
            'seed': '',
            'ten': '',
            'oya': '',
            'hai0': '',
            'hai1': '',
            'hai2': '',
            'hai3': '',
        }
        self.log = self._get_kyoku()
        pass

    def _load_file(self, file):
        with open(file, 'rb') as f:
            data_b = f.read()
            data = data_b.decode('utf-8')
        return data
    
    def _get_tag(self, data, tag):
        soup = BeautifulSoup(data, 'lxml')
        return soup.find_all(tag)
    
    # 用正则表达式匹配所有的对局数据，并转化为实际的对局信息并返回
    def _get_kyoku(self):
        # 存放每一局的数据
        kyoku_all = []
        # 将每一局的数据提取出来
        pattern_segment = re.compile(r'(<INIT.*?>)(.*?)(?=<INIT|$)', re.DOTALL)
        # 判断是否双响
        pattern_agari_2 = re.compile(r'(<AGARI.*?><AGARI.*?>)', re.DOTALL)
        # 分割标签
        pattern_tag = re.compile(r'<.*?/>', re.DOTALL)

        matches_kyoku = pattern_segment.finditer(self.data)
        index = 0
        for match_kyoku in matches_kyoku:
            index += 1
            log = [match_kyoku.group(1)]
            match_agari_2 = pattern_agari_2.findall(match_kyoku.group(2))
            matches_tag = pattern_tag.findall(match_kyoku.group(2))
            if match_agari_2:
                print("出现双响！")
                print("对局文件为：", self.file,"第", index, "局")
                for tag in matches_tag[:-2]:
                    log.append(tag)
                log.append(match_agari_2[0])
            else:
                for tag in matches_tag:
                    log.append(tag)

            
            kyoku_all.append(log)               

        return kyoku_all

    def _get_hai(self, tag):
        hai0 = tag.get('hai0')
        hai0 = hai0.split(',')
        integer_hai0 = [int(x) for x in hai0]
        integer_hai0.sort()
        hand_hai0 = [pai_dict[x] for x in integer_hai0]
        hai1 = tag.get('hai1')
        hai1 = hai1.split(',')
        integer_hai1 = [int(x) for x in hai1]
        integer_hai1.sort()
        hand_hai1 = [pai_dict[x] for x in integer_hai1]
        hai2 = tag.get('hai2')
        hai2 = hai2.split(',')
        integer_hai2 = [int(x) for x in hai2]
        integer_hai2.sort()
        hand_hai2 = [pai_dict[x] for x in integer_hai2]
        hai3 = tag.get('hai3')
        hai3 = hai3.split(',')
        integer_hai3 = [int(x) for x in hai3]
        integer_hai3.sort()
        hand_hai3 = [pai_dict[x] for x in integer_hai3]
        return hand_hai0, hand_hai1, hand_hai2, hand_hai3
    
    def _get_seed(self):
        temp = self.info['seed'].strip().split(',')
        kyoku = kyoku_dict[temp[0]]
        honba = temp[1]
        kyotaku = temp[2]
        dora_indicator = pai_dict[int(temp[-1])]
        return kyoku, honba, kyotaku, dora_indicator
    
    # 获取副露信息
    def _get_naki(self, m):
        color_dict = {
            0: 'm',
            1: 'p',
            2: 's',
        }
        number_dict = {
            0: '123',
            1: '234',
            2: '345',
            3: '456',
            4: '567',
            5: '678',
            6: '789'
        }
        # 转换为16位的2进制数
        m = bin(int(m))[2:].zfill(16)

        bit0_1 = m[-2:]

        bit10_15 = m[:6]

        # bit0~bit1表示来源
        fromWho = int(bit0_1, 2)

        # bit2判断是否为吃
        if m[-3] == '1':
            print("吃")
            # bit10~bit15表示吃的牌
            chi = int(bit10_15, 2)
            # 除以3取整和取余数
            temp, position = divmod(chi, 3)
            # 再除以7取整和取余数
            color, number = divmod(temp, 7)
            print("吃的牌为：", number_dict[number], color_dict[color], "第", position+1, "张")
            if (number == 2 and m[7:8] == '00') or (number == 3 and m[9:10] == '00') or (number == 4 and m[11:12] == '00'):
                print("含有红宝牌")
                
                

        # bit3判断是否为碰
        elif m[-4] == '1':
            print("碰")

        pass

    def print_tehai(self):
        kyoku, honba, kyotaku, dora_indicator = self._get_seed()
        print("局数：", kyoku, "本场：", honba, "供托：", kyotaku, "宝牌指示牌：", dora_indicator)
        print("自家：", self.info['hai0'])
        print("下家：", self.info['hai1'])
        print("对家：", self.info['hai2'])
        print("上家：", self.info['hai3'])

    def _print_kyoku(self, num):
        pattern_draw = re.compile(r'<[TUVW](\d{1,3})/>')
        pattern_discard = re.compile(r'<[DEFG](\d{1,3})/>')
        try:
            log = self.log[num-1]
        except:
            print("对局不存在！")
            return
        print("第", num, "局")
        for tag in log:
            soup = BeautifulSoup(tag, 'lxml')
            if soup.find('init'):
                tehai = self._get_hai(soup.find('init'))
                self.info['hai0'] = tehai[0]
                self.info['hai1'] = tehai[1]
                self.info['hai2'] = tehai[2]
                self.info['hai3'] = tehai[3]
                self.info['oya'] = soup.init.get('oya')
                self.info['ten'] = soup.init.get('ten')
                self.info['seed'] = soup.init.get('seed')
                self.print_tehai()

            elif pattern_draw.match(tag):
                match = pattern_draw.match(tag)
                print(draw_dict[tag[1]], pai_dict[int(match.group(1))])
            elif pattern_discard.match(tag):
                match = pattern_discard.match(tag)
                print(discard_dict[tag[1]], pai_dict[int(match.group(1))])
            elif soup.find('reach'):
                step = soup.find('reach').get('step')
                who = soup.find('reach').get('who')
                if step == '2':
                    ten = soup.find('reach').get('ten')
                    self.info['ten'] = ten
                    print(player_dict[who], "立直成功")
                    print("立直后的点数：", ten)
                else:
                    print(player_dict[who], "宣布立直")
            elif soup.find('dora'):
                print("新宝牌为：", pai_dict[int(soup.dora.get('hai'))])
            elif soup.find('n'):
                m = soup.find('n').get('m')
                result = self._get_naki(m)
                print("副露")
            elif soup.find('agari'):
                who = soup.agari.get('who')
                fromWho = soup.agari.get('fromwho')
                ten_temp = soup.agari.get('ten')
                ten = ten_temp.strip().split(',')
                sc_temp = soup.agari.get('sc')
                sc = sc_temp.strip().split(',')
                # 将sc中的点数转化为整数
                sc = [int(x) for x in sc]

                if who == fromWho:
                    print(player_dict[who], "自摸")
                else:
                    print(player_dict[who], "荣和", player_dict[fromWho])
                print(ten[0],"符",ten[1],"点")
                print("点数变化：")
                for i in range(4):
                    print(player_dict[str(i)], sc[2*i]*100, "+", sc[2*i+1]*100)
            elif soup.find('ryuukyoku'):
                sc_temp = soup.ryuukyoku.get('sc')
                sc = sc_temp.strip().split(',')
                sc = [int(x) for x in sc]

                print("流局")
                if soup.ryuukyoku.get('type'):
                    print("流局原因：", soup.ryuukyoku.get('type'))
                    print("文件：", self.file, "第", num, "局")
        
                print("点数变化：")
                for i in range(4):
                    print(player_dict[str(i)], sc[2*i]*100, "+", sc[2*i+1]*100)

        print("=====================================")

    def print_taikyoku(self, num=0):
        
        if num == 0: # 打印所有对局
            for index, log in zip(range(len(self.log)), self.log):
                self._print_kyoku(index+1)
        else: # 打印指定对局
            self._print_kyoku(num)
            


# 读取文件
path = '../dataset/Tenhou/2011_nan_4_kuitan_aka'
files = os.listdir(path)
pattern = re.compile(r'(\d{4})_nan_4_kuitan_aka_(.*?).log')

for file in files:
    if pattern.match(file):
        file_path = os.path.join(path, file)
        game = taikyoku(file_path)
        # game.print_tehai(num=1)
        # game.print_taikyoku()
        game.print_taikyoku(4)
            

